<a href="https://colab.research.google.com/github/Yangyeeun/20MachineLearning/blob/master/3_06_%EB%AF%B8%EB%8B%88_%EB%B0%B0%EC%B9%98%EC%99%80_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EB%A1%9C%EB%93%9C(Mini_Batch_and_Data_Load).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

데이터를 로드하는 방법과 미니 배치 경사 하강법(Minibatch Gradient Descent)에 대해서 학습하자!


---
# 1. 미니 배치와 배치 크기(Mini Batch and Batch Size)

앞서 배운 다중 선형 회귀에서 사용했던 데이터



```
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])
```

 샘플의 개수는 5개  
 ->전체 데이터를 하나의 행렬로 선언, 전체 데이터에 대해서 경사 하강법을 수행하여 학습  

 샘플의 수가 많아진다면??  
 -> 속도가 느리고 계산량이 많이 필요, 메모리의 한계  

=> 해결! **미니 배치(Mini Batch)**  
전체데이터를 더 작은 단위로 나누어서 해당 단위로 학습하자!  

![대체 텍스트](https://wikidocs.net/images/page/55580/%EB%AF%B8%EB%8B%88%EB%B0%B0%EC%B9%98.PNG)  

미니배치만큼만 가져가 미니 배치에 대한 비용을 계산, 경사하강법 수행  
1 에포크(Epoch): 처음~ 마지막 미니배치까지 전체 데이터에 대한 학습 수행



**에포크(Epoch)** : 전체 훈련 데이터가 학습에 한 번 사용된 주기  
**배치 크기(batch size)** : 미니 배치의 크기 -> 미니 배치의 개수에 따라 결정



배치 경사 하강법: 전체 데이터에 대해서 한 번에 경사 하강법 수행  ->매우 안정, 계산량 多
미니 배치 경사 하강법: 미니 배치 단위로 경사 하강법을 수행 ->훈련 속도 高, 값 조금 헤매기도함



#2. 이터레이션(Iteration)
![대체 텍스트](https://wikidocs.net/images/page/36033/batchandepochiteration.PNG)

**이터레이션** : 한번의 epoch내 이루어지는 W,b의 업데이트 횟수   
전체 데이터= 2,000,  배치 크기= 200-> 이터레이션= 10개  
=> 1 에포크 당 매개변수 업데이트 10번 발생


#3. 데이터 로드하기(Data Load)

Dataset을 정의-> DataLoader에 전달,  
=> 미니 배치 학습, 데이터 셔플(shuffle), 병렬 처리까지 간단히 수행

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
TensorDataset과 DataLoader를 임포트합니다.

from torch.utils.data import TensorDataset # 텐서데이터셋->텐서를 입력으로
from torch.utils.data import DataLoader # 데이터로더

x_train  =  torch.FloatTensor([[73,  80,  75], 
                               [93,  88,  93], 
                               [89,  91,  90], 
                               [96,  98,  100],   
                               [73,  66,  70]])  
y_train  =  torch.FloatTensor([[152],  [185],  [180],  [196],  [142]])

# TensorDataset의 입력으로 사용, dataset으로 저장
dataset = TensorDataset(x_train, y_train)

#모델과 옵티마이저를 설계 (입력= 데이터셋, 미니 배치의 크기, shuffle=True)
#shuffle=True Epoch마다 데이터셋을 섞어서 데이터가 학습되는 순서 변경
# -> 모델이 데이터셋의 순서에 익숙해지는 것을 방지
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

model = nn.Linear(3,1)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5) 

#훈련을 진행
nb_epochs = 20
for epoch in range(nb_epochs + 1):
  for batch_idx, samples in enumerate(dataloader):
    # print(batch_idx)
    # print(samples)
    x_train, y_train = samples
    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.mse_loss(prediction, y_train)

    # cost로 H(x) 계산
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    print('Epoch {:4d}/{} Batch {}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, batch_idx+1, len(dataloader),
        cost.item()
        ))
    

# 임의의 입력 [73, 80, 75]를 선언
new_var =  torch.FloatTensor([[73, 80, 75]]) 
# 입력한 값 [73, 80, 75]에 대해서 예측값 y를 리턴받아서 pred_y에 저장 y= [152],
pred_y = model(new_var) 
print("훈련 후 입력이 73, 80, 75일 때의 예측값 :", pred_y)  # tensor([[154.3850]],